In [1]:
# dependancies
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [2]:
file_dir = '/Users/xenia/Data Bootcamp/Analysis_Projects/capstone/Resources'

athletes_data = pd.read_csv(f'{file_dir}/Final_athlete_clean_data.csv', low_memory=False)
noc_countries = pd.read_csv(f'{file_dir}/Raw_Data/noc_regions.csv', low_memory=False)
countries_gps = pd.read_csv(f'{file_dir}/Raw_Data/concap.csv', low_memory=False)
print('Datasset shape:', athletes_data.shape)
print('Dataset columns:', athletes_data.columns)
athletes_data.head()

Datasset shape: (205879, 12)
Dataset columns: Index(['ID', 'Name', 'Sex', 'Age', 'Height', 'Weight', 'Team', 'NOC', 'Games',
       'Sport', 'Event', 'Medal'],
      dtype='object')


,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Sport,Event,Medal
0,1,A Dijiang,M,24,180.0,80.0,China,CHN,1992 Summer,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23,170.0,60.0,China,CHN,2012 Summer,Judo,Judo Men's Extra-Lightweight,NaN
2,5,Christine Jacoba Aaftink,F,21,185.0,82.0,Netherlands,NED,1988 Winter,Speed Skating,Speed Skating Women's 500 metres,NaN
3,5,Christine Jacoba Aaftink,F,21,185.0,82.0,Netherlands,NED,1988 Winter,Speed Skating,Speed Skating Women's 1000 metres,NaN
4,5,Christine Jacoba Aaftink,F,25,185.0,82.0,Netherlands,NED,1992 Winter,Speed Skating,Speed Skating Women's 500 metres,NaN


In [26]:
athletes_data['Medal'] = athletes_data['Medal'].fillna('No Medal')
pd.reset_option('max_rows')
athletes_data.head()


,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Sport,Event,Medal
0,1,A Dijiang,M,24,180.0,80.0,China,CHN,1992 Summer,Basketball,Basketball Men's Basketball,No Medal
1,2,A Lamusi,M,23,170.0,60.0,China,CHN,2012 Summer,Judo,Judo Men's Extra-Lightweight,No Medal
2,5,Christine Jacoba Aaftink,F,21,185.0,82.0,Netherlands,NED,1988 Winter,Speed Skating,Speed Skating Women's 500 metres,No Medal
3,5,Christine Jacoba Aaftink,F,21,185.0,82.0,Netherlands,NED,1988 Winter,Speed Skating,Speed Skating Women's 1000 metres,No Medal
4,5,Christine Jacoba Aaftink,F,25,185.0,82.0,Netherlands,NED,1992 Winter,Speed Skating,Speed Skating Women's 500 metres,No Medal


In [27]:
map_data = athletes_data.filter(['NOC', 'Games', 'Medal'], axis=1)
map_data

,NOC,Games,Medal
0,CHN,1992 Summer,No Medal
1,CHN,2012 Summer,No Medal
2,NED,1988 Winter,No Medal
3,NED,1988 Winter,No Medal
4,NED,1992 Winter,No Medal
...,...,...,...
205874,POL,1976 Winter,No Medal
205875,POL,2014 Winter,No Medal
205876,POL,2014 Winter,No Medal
205877,POL,1998 Winter,No Medal


In [28]:
map_data['Medal'].value_counts()

No Medal    175785
Bronze       10131
Gold         10118
Silver        9845
Name: Medal, dtype: int64

In [29]:
# Generate our categorical variable lists
map_data_cat = ['Medal']
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)
# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(map_data[map_data_cat]))
# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(map_data_cat)
encode_df

,Medal_Bronze,Medal_Gold,Medal_No Medal,Medal_Silver
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0
...,...,...,...,...
205874,0.0,0.0,1.0,0.0
205875,0.0,0.0,1.0,0.0
205876,0.0,0.0,1.0,0.0
205877,0.0,0.0,1.0,0.0


In [30]:
# Merge one-hot encoded features and drop the originals
map_data = map_data.merge(encode_df, left_index=True, right_index=True).drop(columns=map_data_cat, axis=1)
map_data.head()

,NOC,Games,Medal_Bronze,Medal_Gold,Medal_No Medal,Medal_Silver
0,CHN,1992 Summer,0.0,0.0,1.0,0.0
1,CHN,2012 Summer,0.0,0.0,1.0,0.0
2,NED,1988 Winter,0.0,0.0,1.0,0.0
3,NED,1988 Winter,0.0,0.0,1.0,0.0
4,NED,1992 Winter,0.0,0.0,1.0,0.0


In [33]:
# games participation by country
map_data_grouped = map_data.groupby(['NOC']).nunique().drop(['Medal_Bronze', 'Medal_Gold', 'Medal_No Medal', 'Medal_Silver'], axis=1)
map_data_grouped.reset_index(level=0, inplace=True)
map_data_grouped

,NOC,Games
0,AFG,9
1,AHO,13
2,ALB,10
3,ALG,15
4,AND,21
...,...,...
221,YEM,7
222,YMD,1
223,YUG,23
224,ZAM,13


In [35]:
# medals count by type by country
map_data_medals = map_data.drop(['Games'], axis=1)
map_data_medals_grouped = map_data_medals.groupby(['NOC']).sum()
map_data_medals_grouped.reset_index(level=0, inplace=True)
map_data_medals_grouped

,NOC,Medal_Bronze,Medal_Gold,Medal_No Medal,Medal_Silver
0,AFG,2.0,0.0,52.0,0.0
1,AHO,0.0,0.0,51.0,0.0
2,ALB,0.0,0.0,57.0,0.0
3,ALG,6.0,5.0,466.0,4.0
4,AND,0.0,0.0,135.0,0.0
...,...,...,...,...,...
221,YEM,0.0,0.0,25.0,0.0
222,YMD,0.0,0.0,2.0,0.0
223,YUG,83.0,122.0,1320.0,116.0
224,ZAM,0.0,0.0,127.0,1.0


In [95]:
#combine games and medals
map_data_merged = map_data_grouped.merge(map_data_medals_grouped, on='NOC', how='left')
map_data_merged = map_data_merged.dropna()
print(map_data_merged.shape)
map_data_merged

(226, 6)


,NOC,Games,Medal_Bronze,Medal_Gold,Medal_No Medal,Medal_Silver
0,AFG,9,2.0,0.0,52.0,0.0
1,AHO,13,0.0,0.0,51.0,0.0
2,ALB,10,0.0,0.0,57.0,0.0
3,ALG,15,6.0,5.0,466.0,4.0
4,AND,21,0.0,0.0,135.0,0.0
5,ANG,9,0.0,0.0,245.0,0.0
6,ANT,10,0.0,0.0,115.0,0.0
7,ANZ,2,3.0,0.0,7.0,1.0
8,ARG,34,78.0,72.0,1992.0,70.0
9,ARM,12,8.0,2.0,190.0,5.0


In [96]:
#add country b]names to NOC code and clean data
map_data_countries = noc_countries.merge(map_data_merged, on='NOC', how='left')
map_data_countries = map_data_countries.loc[map_data_countries["NOC"] != 'IOA'] # dropp individuals. 
map_data_countries = map_data_countries.drop(['notes', 'NOC'], axis=1)
map_data_countries = map_data_countries.dropna()
map_data_countries = map_data_countries.sort_values(by=['region'], ascending=True)
map_data_countries_merged =  map_data_countries.groupby(['region']).sum()
map_data_countries_merged.reset_index(level=0, inplace=True)
pd.set_option('display.max_rows', None)
print(map_data_countries_merged.shape)
map_data_countries_merged

(204, 6)


,region,Games,Medal_Bronze,Medal_Gold,Medal_No Medal,Medal_Silver
0,Afghanistan,9.0,2.0,0.0,52.0,0.0
1,Albania,10.0,0.0,0.0,57.0,0.0
2,Algeria,15.0,6.0,5.0,466.0,4.0
3,American Samoa,8.0,0.0,0.0,21.0,0.0
4,Andorra,21.0,0.0,0.0,135.0,0.0
5,Angola,9.0,0.0,0.0,245.0,0.0
6,Antigua,10.0,0.0,0.0,115.0,0.0
7,Argentina,34.0,78.0,72.0,1992.0,70.0
8,Armenia,12.0,8.0,2.0,190.0,5.0
9,Aruba,8.0,0.0,0.0,41.0,0.0


In [57]:
countries_gps_dropped = countries_gps.drop(['CountryCode', 'ContinentName','CapitalName'], axis=1) #select columns
map_data_countries_renamed = map_data_countries_merged.rename({'region': 'CountryName'}, axis=1)  #rename column

In [67]:
pd.set_option('display.max_rows', None) #set display row option to "all rows"
map_data_gps = countries_gps_dropped.merge(map_data_countries_renamed, on='CountryName', how='outer') # add coordinates
map_data_gps = map_data_gps.sort_values(by=['CountryName'], ascending=True) #sort df
print(map_data_gps.shape)
map_data_gps

(259, 8)


,CountryName,CapitalLatitude,CapitalLongitude,Games,Medal_Bronze,Medal_Gold,Medal_No Medal,Medal_Silver
9,Afghanistan,34.516667,69.183333,9.0,2.0,0.0,52.0,0.0
4,Aland Islands,60.116667,19.900000,NaN,NaN,NaN,NaN,NaN
10,Albania,41.316667,19.816667,10.0,0.0,0.0,57.0,0.0
11,Algeria,36.750000,3.050000,15.0,6.0,5.0,466.0,4.0
12,American Samoa,-14.266667,-170.700000,8.0,0.0,0.0,21.0,0.0
13,Andorra,42.500000,1.516667,21.0,0.0,0.0,135.0,0.0
14,Angola,-8.833333,13.216667,9.0,0.0,0.0,245.0,0.0
15,Anguilla,18.216667,-63.050000,NaN,NaN,NaN,NaN,NaN
239,Antarctica,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
245,Antigua,NaN,NaN,10.0,0.0,0.0,115.0,0.0


In [72]:
map_data_gps_coords = map_data_gps[map_data_gps["CapitalLatitude"].isnull()] #select countries with different spelling basing on coorfinates absense
map_data_gps_coords

,CountryName,CapitalLatitude,CapitalLongitude,Games,Medal_Bronze,Medal_Gold,Medal_No Medal,Medal_Silver
245,Antigua,NaN,NaN,10.0,0.0,0.0,115.0,0.0
246,Boliva,NaN,NaN,18.0,0.0,0.0,114.0,0.0
247,Brunei,NaN,NaN,5.0,0.0,0.0,9.0,0.0
248,Curacao,NaN,NaN,13.0,0.0,0.0,51.0,0.0
249,Gambia,NaN,NaN,8.0,0.0,0.0,38.0,0.0
250,Ivory Coast,NaN,NaN,13.0,1.0,1.0,143.0,1.0
251,Micronesia,NaN,NaN,5.0,0.0,0.0,25.0,0.0
252,Saint Kitts,NaN,NaN,6.0,0.0,0.0,39.0,0.0
253,Saint Vincent,NaN,NaN,8.0,0.0,0.0,23.0,0.0
254,Trinidad,NaN,NaN,20.0,20.0,7.0,331.0,7.0


In [83]:
country_name_map = {
    "Antigua and Barbuda": "Antigua",
    "Bolivia": "Boliva",
    "Brunei Darussalam": "Brunei",
    "Curaçao": "Curacao",
    "The Gambia": "Gambia",
    "Cote d'Ivoire": "Ivory Coast",
    "Federated States of Micronesia": "Micronesia",
    "Saint Kitts and Nevis": "Saint Kitts",
    "Saint Vincent and the Grenadines": "Saint Vincent",
    "Trinidad and Tobago": "Trinidad",
    "United Kingdom": "UK",
    "United States": "USA",
    "British Virgin Islands": "Virgin Islands, British",
    "US Virgin Islands": "Virgin Islands, US",

} #create dictionary of correct and wrong names
country_name_map_df = pd.DataFrame.from_dict(country_name_map, orient='index', columns=['foo'])
country_name_map_df.reset_index(level=0, inplace=True)
country_name_map_df = country_name_map_df.rename({'foo': 'CountryName'}, axis=1) 
country_name_map_df


,index,CountryName
0,Antigua and Barbuda,Antigua
1,Bolivia,Boliva
2,Brunei Darussalam,Brunei
3,Curaçao,Curacao
4,The Gambia,Gambia
5,Cote d'Ivoire,Ivory Coast
6,Federated States of Micronesia,Micronesia
7,Saint Kitts and Nevis,Saint Kitts
8,Saint Vincent and the Grenadines,Saint Vincent
9,Trinidad and Tobago,Trinidad


In [86]:
map_data_gps_coords_merged = country_name_map_df.merge(map_data_gps_coords, on='CountryName', how='left')
map_data_gps_coords_merged = map_data_gps_coords_merged.drop(['CountryName','CapitalLatitude','CapitalLongitude'], axis=1)
map_data_gps_coords_merged = map_data_gps_coords_merged.rename({'index': 'CountryName'}, axis=1)
map_data_gps_coords_merged

,CountryName,Games,Medal_Bronze,Medal_Gold,Medal_No Medal,Medal_Silver
0,Antigua and Barbuda,10.0,0.0,0.0,115.0,0.0
1,Bolivia,18.0,0.0,0.0,114.0,0.0
2,Brunei Darussalam,5.0,0.0,0.0,9.0,0.0
3,Curaçao,13.0,0.0,0.0,51.0,0.0
4,The Gambia,8.0,0.0,0.0,38.0,0.0
5,Cote d'Ivoire,13.0,1.0,1.0,143.0,1.0
6,Federated States of Micronesia,5.0,0.0,0.0,25.0,0.0
7,Saint Kitts and Nevis,6.0,0.0,0.0,39.0,0.0
8,Saint Vincent and the Grenadines,8.0,0.0,0.0,23.0,0.0
9,Trinidad and Tobago,20.0,20.0,7.0,331.0,7.0


In [105]:
# combine all data into single DF
map_data_cleaned = map_data_gps.set_index("CountryName").combine_first(map_data_gps_coords_merged.set_index("CountryName")).reset_index()
map_data_final = map_data_cleaned.dropna()
map_data_final[['Games', 'Medal_Bronze', 'Medal_Gold', 'Medal_Gold', 'Medal_No Medal', 'Medal_Silver']] = map_data_final[['Games', 'Medal_Bronze', 'Medal_Gold', 'Medal_Gold', 'Medal_No Medal', 'Medal_Silver']].astype(int)
map_data_final.columns.values[[3, 4, 5,6,7]] = ['Total games', 'Bronze', 'Gold', 'No Medal', 'Silver']
print(map_data_final.shape)
map_data_final

(204, 8)


/Users/xenia/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/pandas/core/frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,CountryName,CapitalLatitude,CapitalLongitude,Total games,Bronze,Gold,No Medal,Silver
0,Afghanistan,34.516667,69.183333,9,2,0,52,0
2,Albania,41.316667,19.816667,10,0,0,57,0
3,Algeria,36.750000,3.050000,15,6,5,466,4
4,American Samoa,-14.266667,-170.700000,8,0,0,21,0
5,Andorra,42.500000,1.516667,21,0,0,135,0
6,Angola,-8.833333,13.216667,9,0,0,245,0
10,Antigua and Barbuda,17.116667,-61.850000,10,0,0,115,0
11,Argentina,-34.583333,-58.666667,34,78,72,1992,70
12,Armenia,40.166667,44.500000,12,8,2,190,5
13,Aruba,12.516667,-70.033333,8,0,0,41,0
